In [1]:
import os
import gc
import numpy as np
import pandas as pd
import json
import argparse
import logging
import boto3
import atexit
import torch
import shutil
import os, errno, time, sys
import ipywidgets as widgets
import torch
from torch.autograd import Variable
import numpy as np
import torch.nn.functional as F
import torchvision
from torchvision import transforms
import torch.optim as optim
from torch import nn
from torch.utils.data import Dataset, DataLoader
from deap import base
from deap import creator
from deap import base, creator, tools, algorithms

from sqlalchemy import create_engine
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import interactive
interactive(True)
import scipy.stats as sts
import scipy as sc
import seaborn as sns
import datetime
from datetime import timedelta
import copy

from torch.cuda import is_available
from catboost import CatBoostClassifier, Pool, cv
from sklearn import metrics
from sklearn.model_selection import train_test_split

from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.metrics import confusion_matrix
from bayes_opt import BayesianOptimization
from datetime import datetime, timedelta
from trains import logger
import operator
import re
import seaborn as sns


import mpld3
from IPython import display
import six
import multiprocessing
import random
import networkx
import time
from IPython import display
import plotly.express as px
import plotly.graph_objects as go
mpld3.enable_notebook()
%matplotlib inline

# data frame with features starts here

In [2]:
df_sorted = pd.read_csv("df_sorted.csv",low_memory=False)

In [3]:
cat_features = ['business_group_name', 'trading_platform',
                'country_code_iso2', 'language_name', 'channel',
                'white_label','identifier_app_source','publisher_name', 'tag_name',
                'affiliate_id','email_provider',
                ]


cyc_features = ['su_dayofweek']

bin_features = ['is_allowed_bonus', 'was_demo'] 

num_features = ['yob',  'su_year']


In [4]:
def to_cyclic_value(x, uniques, func):
    v = round(func(2 * np.pi * x / uniques), 6)
    return v

cyc_features_trig = []
for c in cyc_features:
    for f in [np.sin, np.cos]:
        new_c = c + '_' + str(f.__name__).split('.')[-1]
        cyc_features_trig.append(new_c)

In [5]:
def mapToNum(x,DICT):
    v = DICT[x]
    return v
DICT = dict(zip(cat_features, {}))
cat_features_num = []
for c in cat_features:
    new_c = c+"_num"
    cat_features_num.append(new_c)
    DICT[c] = dict(zip(df_sorted[c].unique(), np.arange(0,df_sorted[c].nunique())))
    df_sorted[new_c] = df_sorted[c].apply(lambda x: mapToNum(x, DICT[c])).copy()



In [6]:
def create_local_dir_if_needed(local_files_dir):
    try:
        os.makedirs(local_files_dir)
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise
        else:
            print("directory %s already exists" % local_files_dir)

In [7]:
all_features = cat_features + bin_features + cyc_features_trig + num_features
all_features2 = cat_features_num + bin_features + cyc_features_trig + num_features
cat_features_indices = list(range(len(cat_features)))
label = 'has_first_deposit_amount'
print(all_features)

['business_group_name', 'trading_platform', 'country_code_iso2', 'language_name', 'channel', 'white_label', 'identifier_app_source', 'publisher_name', 'tag_name', 'affiliate_id', 'email_provider', 'is_allowed_bonus', 'was_demo', 'su_dayofweek_sin', 'su_dayofweek_cos', 'yob', 'su_year']


In [8]:
df_sorted[cat_features_num] = df_sorted[cat_features_num].astype(int).copy()
df_sorted[cat_features] = df_sorted[cat_features].astype(str).copy()
df_sorted[bin_features] = df_sorted[bin_features].astype('float32').copy()
df_sorted[num_features] = df_sorted[num_features].astype('float32').copy()
df_sorted[label] = df_sorted[label].astype('float32').copy()

In [9]:
yob_avg = df_sorted.yob.mean()
df_sorted['yob'].fillna(yob_avg,inplace=True)
df_sorted['was_demo'].fillna(0,inplace=True)
np.sum(df_sorted[all_features].isna())

business_group_name      0
trading_platform         0
country_code_iso2        0
language_name            0
channel                  0
white_label              0
identifier_app_source    0
publisher_name           0
tag_name                 0
affiliate_id             0
email_provider           0
is_allowed_bonus         0
was_demo                 0
su_dayofweek_sin         0
su_dayofweek_cos         0
yob                      0
su_year                  0
dtype: int64

In [10]:
all_cols = all_features.copy()
all_cols.append(label)
all_cols2 = all_features2.copy()
all_cols2.append(label)
all_features = list(dict.fromkeys(all_features))
all_features2 = list(dict.fromkeys(all_features2))
all_cols = list(dict.fromkeys(all_cols))
all_cols2 = list(dict.fromkeys(all_cols2))
print(all_features2)
dataset1 = df_sorted[all_cols].copy()
dataset2 = df_sorted[all_cols2].copy()

['business_group_name_num', 'trading_platform_num', 'country_code_iso2_num', 'language_name_num', 'channel_num', 'white_label_num', 'identifier_app_source_num', 'publisher_name_num', 'tag_name_num', 'affiliate_id_num', 'email_provider_num', 'is_allowed_bonus', 'was_demo', 'su_dayofweek_sin', 'su_dayofweek_cos', 'yob', 'su_year']


In [11]:
train_size = 0.995

N = df_sorted.shape[0]
data_size = df_sorted.shape[0]

permute = np.array(range(N))

train_indexes = np.sort(permute[:int(train_size * N)])
val_indexes = np.sort(permute[int((train_size) * N):])



X, y = dataset1[all_features].values, dataset1[label].values
X2, y2 = dataset2[all_features2].values, dataset2[label].values

df_train = dataset1[all_features].iloc[train_indexes]
df_test = dataset1[all_features].iloc[val_indexes]
df_train2 = dataset2[all_features2].iloc[train_indexes]
df_test2 = dataset2[all_features2].iloc[val_indexes]

X_train, y_train = X[train_indexes, :], y[train_indexes]
X_val, y_val = X[val_indexes, :], y[val_indexes]

X2_train, y2_train = X2[train_indexes, :], y2[train_indexes]
X2_val, y2_val = X2[val_indexes, :], y2[val_indexes]


print("X shape: %s, y shape: %s"%(X.shape, y.shape))
print("y mean is : %.4f" %np.average(y))
print("X_train shape: %s, y_train shape: %s"%(X_train.shape, y_train.shape))
print("y_train mean is : %.4f" %np.average(y_train))
print("X_val shape: %s, y_val shape: %s"%(X_val.shape, y_val.shape))
print("y_val mean is : %.4f" %np.average(y_val))

X shape: (686575, 17), y shape: (686575,)
y mean is : 0.2668
X_train shape: (683142, 17), y_train shape: (683142,)
y_train mean is : 0.2675
X_val shape: (3433, 17), y_val shape: (3433,)
y_val mean is : 0.1180


In [12]:
from sklearn.preprocessing import LabelEncoder
label_encoders = {}
for cat_col in cat_features:
    label_encoders[cat_col] = LabelEncoder()
    df_test[cat_col] = label_encoders[cat_col].fit_transform(df_test[cat_col])
df_test[cat_features].head()    

,business_group_name,trading_platform,country_code_iso2,language_name,channel,white_label,identifier_app_source,publisher_name,tag_name,affiliate_id,email_provider
683142,3,0,120,15,5,2,3,215,237,242,31
683143,2,0,114,4,0,2,3,104,34,35,52
683144,3,1,73,4,5,2,5,215,237,242,31
683145,3,0,69,10,0,2,5,73,55,56,31
683146,6,0,147,4,0,2,5,178,292,272,31


# Variational auto encoder

In [13]:
class FeedForwardNN(nn.Module):
    def __init__(self,  lin_layer_sizes,lin_layer_dropouts,output_size, actiavtions,EncOrDec):
        
        """"
        Parameters
        ----------
        
        lin_layer_sizes: List of integers.
        The size of each linear layer. The length will be equal
        to the total number
        of linear layers in the network.
        
        lin_layer_dropouts: List of floats
        The dropouts to be used after each linear layer.

        output_size: Integer
        The size of the final output.
        
        actiavtions: List of integers.
        1:relu
        2:hardtanh
        3:elu
        4:selu
        5:leaky_relu
        6:celu
        7:rrelu
        8:glu
        9:gelu
        10:logsigmoid
        11:hardshrink
        12:tanhshrink
        13:softsign
        14:softplus
        15:softmin
        16:softmax
        17:softshrink
        18:tanh
        19:sigmoid
        20:linear
        """
        super().__init__()
        self.lin_layers = nn.ModuleList([nn.Linear(lin_layer_sizes[i],
                          lin_layer_sizes[i + 1]) for i in range(len(lin_layer_sizes) - 1)])
        for lin_layer in self.lin_layers:
            nn.init.kaiming_normal_(lin_layer.weight.data)
        # Output Layer
        self.output_layer = nn.Linear(lin_layer_sizes[-1],
                                      output_size)
        nn.init.kaiming_normal_(self.output_layer.weight.data)
        
        # Batch Norm Layers
        
        self.bn_layers = nn.ModuleList([nn.BatchNorm1d(size)
                                            for size in lin_layer_sizes[1:]])

        # Dropout Layers
        
        self.droput_layers = nn.ModuleList([nn.Dropout(size)#, inplace=True
                                      for size in lin_layer_dropouts])
        self.actiavtions = actiavtions
        
    def forward(self, x):


        for lin_layer, dropout_layer, actiavtions in\
            zip(self.lin_layers, self.droput_layers, self.actiavtions):#bn_layer ,self.bn_layers,
                if actiavtions == 1:
                    x = F.relu(lin_layer(x))
                if actiavtions == 2:
                    x = F.hardtanh_(lin_layer(x))
                if actiavtions == 3:
                    x = F.elu_(lin_layer(x))
                if actiavtions == 4:
                    x = F.selu(lin_layer(x),inplace=True)
                if actiavtions == 5:
                    x = F.leaky_relu_(lin_layer(x))
                if actiavtions == 6:
                    x = F.celu(lin_layer(x),inplace=True)
                if actiavtions == 7:
                    x = F.rrelu_(lin_layer(x))
                if actiavtions == 8:
                    x = F.gelu(lin_layer(x))
                if actiavtions == 9:
                    x = F.logsigmoid(lin_layer(x))
                if actiavtions == 10:
                    x = F.hardshrink(lin_layer(x))
                if actiavtions == 11:
                    x = F.tanhshrink(lin_layer(x))
                if actiavtions == 12:
                    x = F.softshrink(lin_layer(x))
                if actiavtions == 13:
                    x = torch.tanh(lin_layer(x))
                if actiavtions == 14:
                    x = torch.sigmoid(lin_layer(x))
                if actiavtions == 15:
                    x = lin_layer(x)

#                 x = bn_layer(x)
                x = dropout_layer(x)

        x = self.output_layer(x)

        return x

In [14]:
batch_size = 1250
dataloader = DataLoader(X2[0:3750,:].astype(float), batch_size, shuffle=True, num_workers=1)
device =  "cpu"#torch.device("cuda") if torch.cuda.is_available() else "cpu"
latent_dim = 90

In [15]:
class Normal(object):
    def __init__(self, mu, sigma, log_sigma, v=None, r=None):
        self.mu = mu.to(device)
        self.sigma = sigma.to(device)  # either stdev diagonal itself, or stdev diagonal from decomposition
        self.logsigma = log_sigma.to(device)
        dim = mu.get_shape()
        if v is None:
            v = torch.FloatTensor(*dim)
        if r is None:
            r = torch.FloatTensor(*dim)
        self.v = v.to(device)
        self.r = r.to(device)
        

class VAE(torch.nn.Module):
    latent_dim = latent_dim
    def __init__(self, encoder, decoder):
        super(VAE, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self._enc_mu = torch.nn.Linear(latent_dim,latent_dim).to(device)
        self._enc_log_sigma = torch.nn.Linear(latent_dim,latent_dim).to(device)

        
    def _sample_latent(self, h_enc):
        """
        Return the latent normal sample z ~ N(mu, sigma^2)
        """
        mu = self._enc_mu(h_enc)
        log_sigma = self._enc_log_sigma(h_enc)
        sigma = torch.exp(log_sigma)
        std_z = torch.from_numpy(np.random.normal(0, 1, size=sigma.size())).float()
        self.z_mean = mu.to(device)
        self.z_sigma = sigma.to(device)
        std_z = std_z.to(device)
        
        
        return mu.to(device) + sigma.to(device) * Variable(std_z, requires_grad=False).to(device)  # Reparameterization trick

    def forward(self, x):
        h_enc = self.encoder(x).to(device)
        z = self._sample_latent(h_enc).to(device)
        return self.decoder(z).to(device)
    

def latent_loss(z_mean, z_stddev):
    mean_sq = z_mean * z_mean
    mean_sq = mean_sq.to(device)
    stddev_sq = z_stddev * z_stddev
    stddev_sq = stddev_sq.to(device)
    return 0.5 * torch.mean(mean_sq.to(device) + stddev_sq.to(device) - torch.log(stddev_sq).to(device) - 1)


# Genearations functions

In [16]:
input_size = len(all_features)

In [17]:

def run_model(lr,
              lin_layer_sizes_enc,lin_layer_dropouts_enc,output_size_enc, actiavtions_enc,
              lin_layer_sizes_dec,lin_layer_dropouts_dec,output_size_dec, actiavtions_dec,
             itter_num,DoPlot):
    BestOfBest = dict()
    N_neurons_enc = []
    N_neurons_dec = []
    f_actiavtions_enc = []
    f_actiavtions_dec = []
    

    
    for N_neurons in lin_layer_sizes_enc:
        N_neurons_enc.append(int(N_neurons))
        
    for N_neurons in lin_layer_sizes_dec:
        N_neurons_dec.append(int(N_neurons))
        
    for f in actiavtions_enc:
        f_actiavtions_enc.append(int(f))
        
    for f in actiavtions_dec:
        f_actiavtions_dec.append(int(f))    
        
    BestLoss = np.inf
    input_dim = len(all_features)-1
    
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


    transform = transforms.Compose([transforms.ToTensor()])


    
    
    
    encoder = FeedForwardNN(N_neurons_enc , lin_layer_dropouts_enc, latent_dim,f_actiavtions_enc,"end")
    decoder = FeedForwardNN(N_neurons_dec , lin_layer_dropouts_dec, input_size,f_actiavtions_dec,"dec")
    encoder = encoder.to(device)
    decoder = decoder.to(device)
    vae = VAE(encoder, decoder)

    
    criterion = nn.MSELoss()

    optimizer = optim.Adam(vae.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-03,
                             weight_decay=0.01, amsgrad=True)

    l = 0

    for epoch in range(itter_num):
        start_time = time.time()

        
        for inputs in dataloader:
            
            
            inputs = Variable(torch.FloatTensor(inputs.float()))
            inputs = inputs.to(device)

            
            dec = vae(inputs)
            dec = dec.to(device)

            
            optimizer.zero_grad()
            ll1 = latent_loss(vae.z_mean, vae.z_sigma)
            ll2 = criterion(dec,inputs)
            loss = ll1 + ll2
            loss.backward()
            optimizer.step()

            l = loss.data.item()
            
              
        end_time = time.time()
        epoch_time = end_time-start_time
                            
        
        
        

        if ll2.detach().cpu().numpy()<BestLoss:
            BestNet = copy.copy(vae)
            BestLoss = ll2.detach().cpu().numpy()
        if np.isnan(ll2.detach().cpu().numpy()):
            BestNet = []
            BestLoss = np.inf
            break
        if DoPlot and epoch%10==0:
            if epoch==0:
                f = go.FigureWidget()
            f.add_trace(go.Scatter(x=[epoch],y=[BestLoss],line=dict(color='firebrick', width=4,
                                      dash='dash'),name=None))
            f.update_layout(showlegend=False)
            display.clear_output(wait=True)
            display.display(f)
            time.sleep(1)
            f.update_layout( yaxis=dict(
                                type="log"
                                ))
    return BestLoss,BestNet
    
        

In [18]:
#limits of neurons per layer
layer_sizes_MIN,layer_sizes_MAX = 0,100
#limits of dropouts 
layer_dropouts_MIN,layer_dropouts_MAX = 0,0.3
#limits of activation functions
actiavtions_MIN,actiavtions_MAX = 1,15

In [19]:
def Build_model(layer_sizes_enc,layer_dropouts_enc,f_actiavtions_enc ,
                layer_sizes_dec,layer_dropouts_dec,f_actiavtions_dec,itter_num,DoPlot):
    
    
    input_dim = 17
    output_size_enc = latent_dim
    lr = 0.016
    
    
    # encoder :
    
    lin_layer_sizes_enc = []
    lin_layer_dropouts_enc = []
    actiavtions_enc=[]
    
    lin_layer_sizes_enc.append(input_dim) 
    lin_layer_dropouts_enc.append(0)
    actiavtions_enc.append(15)
    
    for i,l in enumerate(layer_sizes_enc):
        if l>0:
            lin_layer_sizes_enc.append(np.max([np.min([int(layer_sizes_enc[i]),layer_sizes_MAX]),layer_sizes_MIN]))

            lin_layer_dropouts_enc.append(np.max([np.min([layer_dropouts_enc[i],layer_dropouts_MAX]),
                                                  layer_dropouts_MIN]))

            actiavtions_enc.append(np.max([np.min([int(f_actiavtions_enc[i]),actiavtions_MAX]),actiavtions_MIN]))
        
    # decoder :
    
    lin_layer_sizes_dec = []
    lin_layer_dropouts_dec = []
    actiavtions_dec=[]
    
    lin_layer_sizes_dec.append(latent_dim)
    lin_layer_dropouts_dec.append(0)
    actiavtions_dec.append(15)
    
    for i,l in enumerate(layer_sizes_enc):
        if l>0:
            lin_layer_sizes_dec.append(np.max([np.min([int(layer_sizes_dec[i]),layer_sizes_MAX]),layer_sizes_MIN]))

            lin_layer_dropouts_dec.append(np.max([np.min([layer_dropouts_dec[i],layer_dropouts_MAX]),
                                                 layer_dropouts_MIN]))

            actiavtions_dec.append(np.max([np.min([int(f_actiavtions_dec[i]),actiavtions_MAX]),actiavtions_MIN]))
           
     
    
    output_size_dec = input_dim
    
    
    layer_sizes_enc = []
    layer_dropouts_enc = []
    f_actiavtions_enc = []
    for i,l in enumerate (lin_layer_sizes_enc):
        if l>0:
            layer_sizes_enc.append(l)
            layer_dropouts_enc.append(lin_layer_dropouts_enc[i])
            f_actiavtions_enc.append(actiavtions_enc[i])

    layer_sizes_dec = []
    layer_dropouts_dec = []
    f_actiavtions_dec = []

    for i,l in enumerate (lin_layer_sizes_dec):
        if l>0:
            layer_sizes_dec.append(l)
            layer_dropouts_dec.append(lin_layer_dropouts_dec[i])
            f_actiavtions_dec.append(actiavtions_dec[i])        
    
    
    loss,BestModel = run_model(lr,
                  layer_sizes_enc,layer_dropouts_enc,output_size_enc, f_actiavtions_enc,
                  layer_sizes_dec,layer_dropouts_dec,output_size_dec, f_actiavtions_dec,itter_num,DoPlot)
    
    if loss == np.nan or loss == np.inf or loss == -np.inf:
        runagain = True
        while runagain:
            print(loss,lr)
            lr = lr/10
            loss,BestModel = run_model(lr,
                  layer_sizes_enc,layer_dropouts_enc,output_size_enc, f_actiavtions_enc,
                  layer_sizes_dec,layer_dropouts_dec,output_size_dec, f_actiavtions_dec,itter_num,DoPlot)
            
            if loss == np.nan or loss == np.inf or loss == -np.inf:
                runagain = True
                if lr<1e-12:
                    runagain = False                   
            else:
                runagain = False
        
    return loss,BestModel

In [20]:
def send_pop_to_model(individual):

    layer_sizes_enc = individual[0::6]
    layer_dropouts_enc = individual[1::6]
    f_actiavtions_enc = individual[2::6]
    layer_sizes_dec = individual[3::6]
    layer_dropouts_dec = individual[4::6]
    f_actiavtions_dec = individual[5::6]
    
    loss,BestModel = Build_model(layer_sizes_enc,layer_dropouts_enc,f_actiavtions_enc ,
                    layer_sizes_dec,layer_dropouts_dec,f_actiavtions_dec,30,False)
    
    return loss,

In [21]:
def MateWithBonds(parent1,parent2):
    
    child = tools.cxTwoPoint(parent1,parent2)
    layer_sizes_enc = child[0][0::6]
    layer_dropouts_enc = child[0][1::6]
    f_actiavtions_enc = child[0][2::6]
    layer_sizes_dec = child[0][3::6]
    layer_dropouts_dec = child[0][4::6]
    f_actiavtions_dec = child[0][5::6]
    child = []
    for i,sizes_enc in enumerate(layer_sizes_enc):
        sizes_enc = int(sizes_enc)
        dropouts_enc = layer_dropouts_enc[i]
        actiavtions_enc = int(f_actiavtions_enc[i])
        sizes_dec = int(layer_sizes_dec[i])
        dropouts_dec = layer_dropouts_dec[i]
        actiavtions_dec = int(f_actiavtions_dec[i])
        
        
        sizes_enc = np.min([np.max([sizes_enc,layer_sizes_MIN]),layer_sizes_MAX])
        dropouts_enc = np.min([np.max([dropouts_enc,layer_dropouts_MIN]),layer_dropouts_MAX])
        actiavtions_enc = np.min([np.max([actiavtions_enc,actiavtions_MIN]),actiavtions_MAX])
        sizes_dec = np.min([np.max([sizes_dec,layer_sizes_MIN]),layer_sizes_MAX])
        dropouts_dec = np.min([np.max([dropouts_dec,layer_dropouts_MIN]),layer_dropouts_MAX])
        actiavtions_dec = np.min([np.max([actiavtions_dec,actiavtions_MIN]),actiavtions_MAX])
        
        child.append(sizes_enc)
        child.append(dropouts_enc)
        child.append(actiavtions_enc)
        child.append(sizes_dec)
        child.append(dropouts_dec)
        child.append(actiavtions_dec)
    child = creator.Individual(child)
    return(parent1,child)

In [22]:
def fillpop(df,population):
    n = np.min([df.shape[0],len(population)])
    for i in range(n):
        tmp = df.individuals.iloc[i].replace('[','')
        tmp = tmp.replace(']','')
        tmp = tmp.split(',')
        tmp2 = []
        for l in tmp:
            tmp2.append(float(l))
        population[i] = creator.Individual(tmp2)
        population[i].fitness.values = df.fitness.iloc[i],
    
    return(population)

In [23]:
def run_model_after_ga(individual):

    layer_sizes_enc = individual[0::6]
    layer_dropouts_enc = individual[1::6]
    f_actiavtions_enc = individual[2::6]
    layer_sizes_dec = individual[3::6]
    layer_dropouts_dec = individual[4::6]
    f_actiavtions_dec = individual[5::6]
    
    loss,BestModel = Build_model(layer_sizes_enc,layer_dropouts_enc,f_actiavtions_enc ,
                    layer_sizes_dec,layer_dropouts_dec,f_actiavtions_dec,3000,True)
    
    return loss,BestModel 

# Genetic algorithm starts here

In [25]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

population_size = 30
num_generations = 100
max_layers = 5
gene_length = 1
toolbox = base.Toolbox()

In [26]:

if __name__ == "__main__":
    toolbox.register("attr_layer_sizes_enc", random.randint, layer_sizes_MIN,layer_sizes_MAX)
    toolbox.register("attr_layer_dropouts_enc", random.uniform, layer_dropouts_MIN,layer_dropouts_MAX)
    toolbox.register("attr_actiavtions_enc", random.randint,actiavtions_MIN,actiavtions_MAX )
    toolbox.register("attr_layer_sizes_dec", random.randint, layer_sizes_MIN,layer_sizes_MAX)
    toolbox.register("attr_layer_dropouts_dec", random.uniform,layer_dropouts_MIN,layer_dropouts_MAX)
    toolbox.register("attr_actiavtions_dec", random.randint, actiavtions_MIN,actiavtions_MAX)
    toolbox.register("individual", tools.initCycle, creator.Individual,
    (toolbox.attr_layer_sizes_enc, toolbox.attr_layer_dropouts_enc,toolbox.attr_actiavtions_enc,
    toolbox.attr_layer_sizes_dec,toolbox.attr_layer_dropouts_dec,toolbox.attr_actiavtions_dec), n=max_layers)
    toolbox.register('population', tools.initRepeat, list, toolbox.individual)
    toolbox.register("MateWithBonds",MateWithBonds)
    toolbox.register("mate", toolbox.MateWithBonds)
    toolbox.register('mutate', tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
    toolbox.register('select', tools.selTournament,tournsize=5)
    toolbox.register('evaluate', send_pop_to_model)

    population = toolbox.population(n = population_size)
    
    

In [27]:

hof = tools.HallOfFame(3)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.average)
stats.register("std", np.std)
stats.register("min", min)
stats.register("max", max)

# first run

In [27]:
CSVsPath = '/home/octopus/projects/sandbox/GA_VAE/CSVs_4/'

create_local_dir_if_needed(CSVsPath)

r = algorithms.eaSimple(population, toolbox, cxpb = 0.33, mutpb = 0.25, ngen = 1,stats=stats,
                                 halloffame = hof,verbose = True)

filename = "gen_0.csv"
fitness = []
for i,pop in enumerate(population):
    fitness.append(population[i].fitness.values[0])
df = pd.DataFrame({"individuals":population,"fitness":fitness})
df.to_csv(CSVsPath + filename)    

directory /home/octopus/projects/sandbox/GA_VAE/CSVs_4/ already exists
inf 0.016
inf 0.016
inf 0.0016
inf 0.00016
inf 1.6000000000000003e-05
inf 1.6000000000000004e-06
inf 1.6000000000000003e-07
inf 1.6000000000000004e-08
inf 1.6000000000000005e-09
inf 1.6000000000000004e-10
inf 1.6000000000000003e-11
inf 1.6000000000000003e-12
inf 0.016
inf 0.0016
inf 0.00016
inf 1.6000000000000003e-05
inf 1.6000000000000004e-06
inf 1.6000000000000003e-07
inf 1.6000000000000004e-08
inf 1.6000000000000005e-09
inf 1.6000000000000004e-10
inf 1.6000000000000003e-11
inf 1.6000000000000003e-12
inf 0.016
inf 0.0016
inf 0.00016
inf 1.6000000000000003e-05
inf 1.6000000000000004e-06
inf 1.6000000000000003e-07
inf 1.6000000000000004e-08
inf 1.6000000000000005e-09
inf 1.6000000000000004e-10
inf 1.6000000000000003e-11
inf 1.6000000000000003e-12
inf 0.016
inf 0.0016
inf 0.00016
inf 1.6000000000000003e-05
inf 1.6000000000000004e-06
inf 1.6000000000000003e-07
inf 1.6000000000000004e-08
inf 1.6000000000000005e-09
inf 

/home/octopus/virtualenv/lib/python3.6/site-packages/numpy/core/_methods.py:117: RuntimeWarning:

invalid value encountered in subtract



inf 0.016
1  	15    	66500.5	139050	(2563.775146484375,)	(471036.34375,)


In [28]:
CSVsPath = '/home/octopus/projects/sandbox/GA_VAE/CSVs_4/'
gen = 7
filename = "gen_"+str(gen)+".csv"
df = pd.read_csv(CSVsPath + filename)
population = fillpop(df,population)
Q = stats.compile(population)
stats_df = pd.DataFrame(Q)
stats_df['generation'] = gen
stats_df.set_index('generation',inplace=True)
stats_df.head()

/home/octopus/virtualenv/lib/python3.6/site-packages/numpy/core/_methods.py:117: RuntimeWarning:

invalid value encountered in subtract



,avg,std,min,max
generation,,,,
7,inf,NaN,1084.560059,inf


In [29]:

f = go.FigureWidget()


for gen in range(gen,num_generations):
    Q = stats.compile(population)
    f.add_trace(go.Scatter(x=[gen],y=[Q['min'][0]], mode='markers',line=dict(color='firebrick', width=4,
                          dash='dash'),name=None))
    f.update_layout(showlegend=False)
    display.clear_output(wait=True)
    display.display(f)
    time.sleep(1)
    f.update_layout( yaxis=dict(
                        type="log"
                        ))
    filename = "gen_"+str(gen)+".csv"
    df = pd.read_csv(CSVsPath + filename)
    population = fillpop(df,population)
    r = algorithms.eaSimple(population, toolbox, cxpb = 0.33, mutpb = 0.25, ngen = 1,stats=stats,
                                 halloffame = hof,verbose = True)
    fitness = []
    for i,pop in enumerate(population):
        fitness.append(population[i].fitness.values[0])
    df = pd.DataFrame({"individuals":population,"fitness":fitness})
    filename = "gen_"+str(gen+1)+".csv"
    df.to_csv(CSVsPath + filename)

FigureWidget({
    'data': [{'line': {'color': 'firebrick', 'dash': 'dash', 'width': 4},
              'mode':…

gen	nevals	avg    	std    	min                 	max                 
0  	0     	1017.46	567.646	(809.1901245117188,)	(3185.023193359375,)
1  	17    	23222.9	85564.3	(809.1901245117188,)	(405921.0625,)      


In [ ]:

df = pd.read_csv(filename)
# df.fitness = df.fitness.apply(lambda x: float(x.replace('(','').replace(')','').replace(',','')))
df.sort_values(by='fitness',inplace=True,ascending=False)

df.drop_duplicates(subset=['fitness'],inplace=True)
# df.reset_index(inplace=True)
population = fillpop(df,population)
BestModel = population[0]

df.to_csv("gen.csv")
df.head(300)

In [ ]:
loss,BestModel = run_model_after_ga(BestModel)

In [ ]:
import pickle
fitness = []
for pop in population:
    fitness.append(pop.fitness.wvalues[0])
    
cp = dict(population=population,fitness=fitness, generation=1, halloffame=hof,
                       rndstate=random.getstate())

with open("checkpoint_name.pkl", "wb") as cp_file:
    pickle.dump(cp, cp_file)

In [ ]:
with open("checkpoint_name.pkl", "r") as cp_file:
    cp = pickle.load("checkpoint_name.pkl")

In [ ]:
cp['fitness']

In [ ]:
for pop in population:
    print(vars(stats))

In [ ]:
inputs = dataloader.dataset[100]

In [ ]:
inputs = Variable(torch.FloatTensor(inputs))
inputs = inputs.to(device)

In [ ]:
outputs = BestModel(inputs.reshape(1,17))

In [ ]:
outputs.detach().cpu().numpy()

In [ ]:
out_cat = outputs[0]
out_cat.detach().cpu().numpy()

In [ ]:

true = []
pred = []
for i,cat in enumerate(cat_features):
    m = np.max(list(DICT[cat].values()))
    true.append(list(DICT[cat].keys())[list(DICT[cat].values()).index(int(inputs[i]))])
    pred.append(list(DICT[cat].keys())[list(DICT[cat].values()).index(int(np.min([out_cat[i],m])))])
    print(list(DICT[cat].keys())[list(DICT[cat].values()).index(int(inputs[i]))],"\t : \t",
        list(DICT[cat].keys())[list(DICT[cat].values()).index(int(np.min([out_cat[i],m])))])
    

In [ ]:
for i,f in enumerate(all_features[len(cat_features):]):
    m = len(cat_features)+i
    true.append(inputs[m].detach().cpu().numpy())
    pred.append(out_cat[m].detach().cpu().numpy())
    print(f,":\ninput : ",inputs[m].detach().cpu().numpy()," - output : ",out_cat[m].detach().cpu().numpy())


In [ ]:
len(true)

In [ ]:
df_res = pd.DataFrame({"feature":all_features,"input":true,"output":pred})
df_res.head(17)

In [ ]:
lat_vec = []
for x in dataloader:
    for y in x:
        inputs = Variable(torch.FloatTensor(y.float()))
        inputs = inputs.to(device)
        l_vec = BestNet.encoder(inputs)
        lat_vec.append(l_vec.detach().cpu().numpy())
lat_vec    

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca.fit(lat_vec)      

In [ ]:
xx = []
yy = []
for x,y in pca.transform(lat_vec):
    xx.append(x)
    yy.append(y)
    
_=plt.scatter(xx,yy)    